# Face clustering with DLIB

In [32]:
import dlib
from os import listdir, chdir, path, getcwd
from skimage import io
import time
import numpy as np
from sklearn.cluster import DBSCAN

SHAPE_PREDICTOR_MODEL_PATH = "../Dlib/Models/shape_predictor_5_face_landmarks.dat"
FACE_RECOGNITION_MODEL_PATH = "../Dlib/Models/dlib_face_recognition_resnet_model_v1.dat"
DATA_FOLDER = "../Data/Recognition/"
SAVE_FOLDER = DATA_FOLDER + "Clustering/"

In [35]:
files = listdir(IMAGES_FOLDER, )
image_files = [f for f in files if f.endswith('.jpg')]

In [46]:
detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor(SHAPE_PREDICTOR_MODEL_PATH)
facerec = dlib.face_recognition_model_v1(FACE_RECOGNITION_MODEL_PATH)
face_descriptor = list()
detected_faces = list()
count = 0

# win = dlib.image_window()

for f in image_files:
    
    count += 1
    
    print("{}. Processing file {}".format(count, f))
    img = io.imread(path.join(IMAGES_FOLDER, f))
    
    # win.clear_overlay()
    # win.set_image(img)
    
    dets = detector(img, 1)
    print("Number of faces detected: {}".format(len(dets)))
    
    for i, d in enumerate(dets):
        print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
            i, d.left(), d.top(), d.right(), d.bottom()))
        top = max(0, d.top())
        bottom = min(img.shape[0], d.bottom())
        left = max(0, d.left())
        right = min(img.shape[1], d.right())
        face = img[top:bottom, left:right,:]
        detected_faces.append(face)
        shape = sp(img, d)
        face_descriptor.append(np.array(facerec.compute_face_descriptor(img, shape, 100)))
        
        # win.add_overlay(d)
        # win.add_overlay(shape)
    print("")             

    face_descriptor = np.vstack(face_descriptor)

1. Processing file bald_guys.jpg
Number of faces detected: 24
Detection 0: Left: 1457 Top: 1343 Right: 1586 Bottom: 1472
Detection 1: Left: 1067 Top: 1295 Right: 1253 Bottom: 1481
Detection 2: Left: 56 Top: 77 Right: 242 Bottom: 263
Detection 3: Left: 1047 Top: 758 Right: 1232 Bottom: 944
Detection 4: Left: 551 Top: 428 Right: 737 Bottom: 614
Detection 5: Left: 1823 Top: 438 Right: 1931 Bottom: 546
Detection 6: Left: 35 Top: 1274 Right: 221 Bottom: 1460
Detection 7: Left: 786 Top: 821 Right: 941 Bottom: 976
Detection 8: Left: 64 Top: 700 Right: 218 Bottom: 855
Detection 9: Left: 64 Top: 391 Right: 218 Bottom: 546
Detection 10: Left: 322 Top: 442 Right: 476 Bottom: 597
Detection 11: Left: 1658 Top: 703 Right: 1748 Bottom: 793
Detection 12: Left: 340 Top: 1232 Right: 563 Bottom: 1455
Detection 13: Left: 861 Top: 407 Right: 1047 Bottom: 593
Detection 14: Left: 67 Top: 999 Right: 196 Bottom: 1128
Detection 15: Left: 390 Top: 811 Right: 613 Bottom: 1034
Detection 16: Left: 342 Top: 56 Right

In [47]:
cl = DBSCAN(eps=0.48, min_samples=1, metric='euclidean')
people = cl.fit_predict(face_descriptor)
count_by = np.bincount(people)
persons = np.nonzero(count_by)[0]

In [48]:
print("Imagen => individuo")
for i,j in zip(range(0, len(people)), people):
    if (i < 10):
        print(" " + str(i) + " => " + str(j))
    else:
        print(str(i) + " => " + str(j))

Imagen => individuo
 0 => 0
 1 => 0
 2 => 1
 3 => 2
 4 => 1
 5 => 0
 6 => 1
 7 => 2
 8 => 3
 9 => 1
10 => 3
11 => 0
12 => 2
13 => 3
14 => 2
15 => 3
16 => 3
17 => 2
18 => 1
19 => 2
20 => 1
21 => 0
22 => 0
23 => 3


In [49]:
print("Individuo: número de veces que aparece")
for i,j in zip(persons, count_by):
    if (i < 10):
        print(" " + str(i) + ": " + str(j))
    else:
        print(str(i) + ": " + str(j))

Individuo: número de veces que aparece
 0: 6
 1: 6
 2: 6
 3: 6


In [51]:
# win = dlib.image_window()
for person in persons:
    ind_faces = np.where(people == person)[0]
    faces = [detected_faces[i] for i in ind_faces]
    faces_dim = np.asarray([faces[i].shape for i in range(0,len(faces))])

    max_rows = max(faces_dim[:,0])
    max_cols = max(faces_dim[:,1])
    pad_faces = np.asarray([np.lib.pad(faces[i], 
                            (
                                (0, max_rows - faces[i].shape[0]),
                                (0, max_cols - faces[i].shape[1]),
                                (0,0)
                            ), 
                            'constant', 
                            constant_values = 255) 
                 for i in range(0, len(faces))
                ])
    
    concat_faces = np.hstack(pad_faces) 
    io.imsave(SAVE_FOLDER + "person_" + str(person) + ".jpg", concat_faces)
    
    # win.clear_overlay()
    # win.set_image(concat_faces)
    # time.sleep(2)